In [ ]:
# this scripts reads the .txt file produced by joint_to_txt.cpp macros
# using following data: amplitude, time, z-coordinate of pulses
# and produces the time/z/amplitude distribution with fitted zenith angle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

ns = 5 # ADC count length
C_w = 0.225 # speed of light in water, m/ns
C = 0.299 # speed of light in vacuum, m/ns

filename = "out/thres10000.txt"
df = pd.read_csv(filename, sep='\t', header=0)
df['T']*=5
# adding column with time*C_w for fit
df['T_norm'] = df['T']*C_w

# sorting time

df.sort_values(by=["j", "T"], inplace=True)
df.reset_index(drop=True, inplace=True)

#print(max_charge_idx)

print(df.head(40))
#print(len(df['j']))
#print(np.max(df['T']))
#print(np.min(df['T']))

In [ ]:
# assigning the flags for scattered and direct light
# calculating dt and dz for all the pulses

df['dt'] = df.groupby(['j'])['T'].diff().fillna(0)
df['dz'] = df.groupby(['j'])['z'].diff().fillna(0)

# scatter light by time-of-flight
df['scatter_flag'] = df['dt'] > abs(df['dz']) / C_w

# taking times of peak pulses for each event
peak_times = df['T'][(df.groupby(['j'])['Q'].idxmax().values)].values

# fix mismatching flags for loud pulses
df['scatter_flag'][(df.groupby(['j'])['Q'].idxmax().values)] = False

df.head(30)

In [ ]:
# functions library
import numpy as np
from scipy.optimize import curve_fit
from scipy.spatial.transform import Rotation as R

def hyperbolic_func(x, a, b, c, d, e, theta, xy_out = False):
    '''Rotated hyperbola function.
    Returns rotated x,y or y points depends on flag.
    Theta in radians'''
    R = [[np.cos(theta), -np.sin(theta)],
         [np.sin(theta),  np.cos(theta)]]
    y = a / (x - b) + c + d * x + e * x ** 2
    x_rot = x * np.cos(theta) - y * np.sin(theta)
    y_rot = x * np.sin(theta) + y * np.cos(theta)
    if xy_out:
        return x_rot, y_rot
    else:
        return y_rot

def fit_hyperbola(df):
    '''Just fit '''
    x = df['T_norm'].values
    y = df['z'].values
    weights = df['Q'].values
    params, pcov = curve_fit(hyperbolic_func, x, y, 
                          sigma=1/weights, 
                          p0=[1, 0, 0, 0, 0, 0.5],
                          maxfev = 100000)

    return params, pcov

def r_square(df, params):
    '''Calculates coefficient of determination
    Needed only for estimation of fit quality.
    TODO works strange'''
    x = df['T_norm']
    y = df['z']
    y_fit = hyperbolic_func(x, *params)
    # residual sum of squares
    ss_res = np.sum((y - y_fit) ** 2)
    # total sum of squares
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    # r-squared
    return 1 - (ss_res / ss_tot)

In [ ]:
# Plot time / height / amplitude distribution
# Fit
cut = 50 # to plot first N events
for j in np.unique(df['j'])[:cut]:
    df_event = df[df['j'] == j]
    direct = df_event[df_event['scatter_flag'] == False]
    scatter = df_event[df_event['scatter_flag'] == True]
    
    t_d = direct['T']
    z_d = direct['z']
    q_d = direct['Q']
    t_s = scatter['T']
    z_s = scatter['z']
    q_s = scatter['Q']

    model_z = np.linspace(0,500,50)
    model_t = np.linspace(0,4000,50)
    model_tnorm = model_t * C_w
    params, pcov = fit_hyperbola(direct)
    fig, ax = plt.subplots()
    scatter_direct = ax.scatter(t_d, z_d, c=q_d, 
                                cmap='Greens', 
                                norm=matplotlib.colors.LogNorm(
                                    vmin = 10,
                                    vmax = 10**5), 
                                edgecolors= "green",
                                )
    
    scatter_scatter= ax.scatter(t_s, z_s, c=q_s, 
                                cmap='Reds', 
                                norm=matplotlib.colors.LogNorm(
                                vmin = 10,
                                vmax = 10**5), 
                                edgecolors= "red",
                                )
    fit = ax.plot(model_t, hyperbolic_func(
                                           model_tnorm,
                                           *params
                                        ),
                    label='event %d \n\u03F4 fit=%.2f\u00B0 \nR2=%f' % 
                          (j,params[5],r_square(direct,params))      
                 )
    ax.set_xlabel('Time, ns')
    ax.set_ylabel('Height, m')
    ax.set_ylim(0,600)
    plt.colorbar(scatter_direct)
    plt.grid()
    plt.legend(loc='upper left')
#    plt.savefig('plots/'+str(j)+'.png')
    plt.show()
    plt.clf()
    

In [ ]:
# plot amplitude histogram
fig, ax = plt.subplots()
ax.hist(df['Q'], bins=100, edgecolor='black', log=True)

ax.set_xlabel('Amplitude')
ax.set_ylabel('Counts')

plt.show()

In [ ]:
# plot time diff histogram
fig, ax = plt.subplots()
timedeltas = 
ax.hist(df['Q'], bins=100, edgecolor='black', log=True)

ax.set_xlabel('Amplitude')
ax.set_ylabel('Counts')

plt.show()

In [ ]:
# not used
def scatter_filter(t,z):
    C = 0.299 # m/ns
    C_w = 0.2148# m/ns
    coeff = 1
    i=0
    j=1
    t_res = []
    z_res = []
#    print('t_diff, z_diff, t_diff*C, z[i], z[i+1], t[i], t[i+1]')
    while i<len(t)-1:
        if i+j >= len(t):
            break
        t_diff = abs(t[i] - t[i+j])
        z_diff = abs(z[i] - z[i+j])

#        if t_diff <= z_diff/C and 0 < z_diff <= 45:# and t_diff <= z_diff/(C_w*np.sin(np.pi-np.radians(82))):
        if (t_diff <= z_diff/(C_w*np.sin(np.pi-np.radians(82)))*coeff) and 0 <= z_diff <= 45:
            t_res.append(t[i])
            z_res.append(z[i])
            print(str(t[i])+'\t'+str(z[i])+' HIT')
#            j=1
        elif z_diff == 0:
            t_res.append(t[i])
            z_res.append(z[i])
            print(str(t[i])+'\t'+str(z[i])+' HIT')
            i+=1
        else:
            print(str(t[i])+'\t'+str(z[i])+' SCAT '+str(t_diff)+' > '+str(z_diff/(C_w*np.sin(np.pi-np.radians(82)))*coeff))
#            j+=1
        i+=1
    return t_res,z_res
    